In [1]:
import os
import sys
import json
import numpy as np
from tqdm import tqdm
sys.path.append("..")

from torch import nn
from torch.optim import Adam
from src.movientities.models.char_lstm import CharBilstm
from src.movientities.utils.processors import NerPreProcessor
from src.movientities.char_bilstm_trainer import CharBilstmTrainer
from src.movientities.data.build_dataset import CharCorpus, BuildData

In [2]:
dataset = CharCorpus(
    input_folder=f"/home/kemalaraz/Desktop/MovieEntityRecognizer/data/modified/mitmovies_tab_format",
    min_word_freq=3,
    batch_size=64,
)
print(f"Train set: {len(dataset.train_dataset)} sentences")
print(f"Test set: {len(dataset.test_dataset)} sentences")

Train set: 9775 sentences
Test set: 2443 sentences


In [3]:
bilstm = CharBilstm(
    input_dim=len(dataset.word_field.vocab),
    embedding_dim=300,
    char_emb_dim=25,
    char_input_dim=len(dataset.char_field.vocab),
    char_cnn_filter_num=5,
    char_cnn_kernel_size=3,
    hidden_dim=64,
    output_dim=len(dataset.tag_field.vocab),
    lstm_layers=2,
    emb_dropout=0.5,
    cnn_dropout=0.25,
    lstm_dropout=0.1,
    fc_dropout=0.25,
    word_pad_idx=dataset.word_pad_idx,
    char_pad_idx=dataset.char_pad_idx
)
bilstm.init_embeddings(
    char_pad_idx=dataset.char_pad_idx,
    word_pad_idx=dataset.word_pad_idx,
    pretrained=None,
    freeze=True
)
print(f"The model has {bilstm.count_parameters():,} trainable parameters.")
print(bilstm)

The model has 1,028,749 trainable parameters.
CharBilstm(
  (embedding): Embedding(2244, 300, padding_idx=1)
  (emb_dropout): Dropout(p=0.5, inplace=False)
  (char_emb): Embedding(39, 25, padding_idx=1)
  (char_cnn): Conv1d(25, 125, kernel_size=(3,), stride=(1,), groups=25)
  (cnn_dropout): Dropout(p=0.25, inplace=False)
  (lstm): LSTM(425, 64, num_layers=2, dropout=0.1, bidirectional=True)
  (fc_dropout): Dropout(p=0.25, inplace=False)
  (fc): Linear(in_features=128, out_features=26, bias=True)
)


In [4]:
ner = CharBilstmTrainer(
  model=bilstm,
  data=dataset,
  optimizer_cls=Adam,
  loss_fn_cls=nn.CrossEntropyLoss,
  log_file="char_bilstm_vanilla"
)
ner.train(20)

Epoch: 01 | Epoch Time: 0m 28s
	Trn Loss: 1.417 | Trn Acc: 67.14%
	Val Loss: 0.783 | Val Acc: 81.16% | Val Precision: 59.04% | Val Recall: 60.19% | Val F1 Macro: 56.12% | Val F1 Micro: 81.51%
Epoch: 02 | Epoch Time: 0m 32s
	Trn Loss: 0.521 | Trn Acc: 86.97%
	Val Loss: 0.497 | Val Acc: 87.46% | Val Precision: 76.16% | Val Recall: 73.35% | Val F1 Macro: 72.54% | Val F1 Micro: 86.87%
Epoch: 03 | Epoch Time: 0m 36s
	Trn Loss: 0.367 | Trn Acc: 90.63%
	Val Loss: 0.461 | Val Acc: 88.19% | Val Precision: 76.66% | Val Recall: 73.77% | Val F1 Macro: 72.66% | Val F1 Micro: 87.32%
Epoch: 04 | Epoch Time: 0m 41s
	Trn Loss: 0.309 | Trn Acc: 92.06%
	Val Loss: 0.410 | Val Acc: 89.59% | Val Precision: 79.77% | Val Recall: 77.22% | Val F1 Macro: 76.47% | Val F1 Micro: 88.60%
Epoch: 05 | Epoch Time: 0m 40s
	Trn Loss: 0.272 | Trn Acc: 92.97%
	Val Loss: 0.388 | Val Acc: 90.01% | Val Precision: 78.03% | Val Recall: 76.75% | Val F1 Macro: 75.34% | Val F1 Micro: 88.90%
Epoch: 06 | Epoch Time: 0m 44s
	Trn Loss